# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You are permitted to write additional helper functions, or use additional instance variables so long as the previously described functions work as required.  

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [26]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import *

#### Define Spark Context
<span style="color:red">*Please don't modify the below cell*</span>

In [2]:
sc
sqlContext = SQLContext(sc)

### Student Section - Please compete all the functions below

#### Function to return GT Username

In [3]:
def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'icanlapan3'

#### Function to load data

In [39]:
def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP storage bucket#

    df = spark.read.option ("inferSchema" , "true").option ("header" , "true") \
              .csv(gcp_storage_path)
        
    # Convert to decimal trip_distance, fare_amount, tip_amount, tolls_amount
    
    df = df.withColumn("trip_distance",df["trip_distance"].cast('Decimal'))\
           .withColumn("fare_amount",df["fare_amount"].cast('Decimal'))\
           .withColumn("tip_amount",df["tip_amount"].cast('Decimal'))\
           .withColumn("tolls_amount",df["tolls_amount"].cast('Decimal'))
    
    
    ################################################################
        
    return df

#### Function to exclude trips that don't have a pickup location

In [41]:
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to exclude trips with no pickup locations               #
    # Note: Exclude  nulls, blanks and zeros                       #        
    ################################################################
    
    df.createOrReplaceTempView("data")
    df2 = spark.sql("""
                    select *
                    from data 
                    where ((PULocationID is not null) or (PULocationID != ""))
                    """)
    return df2


#### Function to exclude trips with no distance

In [73]:
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    ################################################################
    # code to exclude trips with no trip distances                 #
    # Note: Exclude  nulls, blanks and zero                        #        
    ################################################################
    
    df.createOrReplaceTempView("data")
    df2 = spark.sql("""
                    select *
                    from data 
                    where ((trip_distance is not null) or (trip_distance != "") or (trip_distance > 0))
                    """)
    
    return df2

#### Function to include fare amount between the range of 20 to 60 Dollars

In [44]:
def include_fare_range(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars (including 20 and 60 dollars)               #        
    ################################################################
    
    df.createOrReplaceTempView("data")
    df2 = spark.sql("""
                    select *
                    from data 
                    where fare_amount >= 20 
                    and fare_amount <= 60
                    """)
    return df2

#### Function to get the highest tip amount

In [65]:
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get the highest tip                                  #
    #                                                              #        
    ################################################################
    
    df.createOrReplaceTempView("data")
    df2 = spark.sql("""
                    select round(max(tip_amount),2) as max_tip
                    from data
                    """)

    return df2.collect()[0]['max_tip']

#### Function to get total toll amount

In [71]:
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get total toll amount                                #
    ################################################################
    
    df.createOrReplaceTempView("data")
    df2 = spark.sql("""
                    select round(sum(tolls_amount),2) as total_toll
                    from data
                    """)

    return df2.collect()[0]['total_toll']

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata_2019-01.csv 

In [40]:
gcp_storage_path = "gs://icanlapan3/yellow_tripdata_2019-01.csv"
df = load_data(gcp_storage_path)
df.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: decimal(10,0) (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: decimal(10,0) (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: decimal(10,0) (nullable = true)
 |-- tolls_amount: decimal(10,0) (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



#### Print total numbers of rows in the dataframe

In [6]:
df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [31]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()

7667792

#### Print total number of rows in the dataframe after exclude trips with no distance

In [74]:
df_no_trip_distance = exclude_no_tripdistance(df)
df_no_trip_distance.count()

7667792

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [45]:
df_include_fare_range = include_fare_range(df)
df_include_fare_range.count()

1011593

#### Print the highest tip amount

In [70]:
max_tip = get_highest_tip(df)
print(max_tip)

787


#### Print the total toll amount

In [72]:
total_toll = get_total_toll(df)
print(total_toll)

2525415
